In [1]:
using Plots, Measures

include("../src/LoadData.jl")
include("../src/MarginalPosterior.jl")
include("../src/PMMH.jl")
include("../src/Support.jl")

# Load data
Yin = loadData("NZCOVID_1APR2024");

# Use only the first 100 days when fitting the model
Y = Yin[1:100,:];

In [2]:
function naiveModel(θ, Y::DataFrame, opts::Dict)
    
    # Extract frequently used options
    T = opts["T"]
    N = opts["N"]
    L = opts["L"]
    ω = opts["ω"]
    delayDist = opts["delayDist"]
    h = opts["forecastingHorizon"]
    
    # Initialise output matrices
    R = zeros(N, T+h)
    I = zeros(N, T+h)
    W = ones(N, T)
    
    # And predictive values
    μ = zeros(N, T+h) # Store expected cases to avoid resampling issues
    C = zeros(N, T+h)
    
    # Sample from initial distributions
    R[:,1] = rand(opts["pR0"], N)
    I[:,1] = rand(opts["pI0"], N)
    
    # Run the filter
    for tt = 2:T
        
        # Project according to the state-space model
        R[:,tt] = exp.(rand.(Normal.(log.(R[:,tt-1]), θ[1])))
        Λ = sum(I[:, (tt-1):-1:1] .* ω[1:(tt-1)]', dims=2) ./ sum(ω[1:(tt-1)])
        I[:,tt] = rand.(Poisson.(R[:,tt] .* Λ))
                    
        # Weight according to the observation model
        μt = sum(I[:,(tt-1):-1:1] .* delayDist[1:(tt-1)]', dims=2) ./ sum(delayDist[1:(tt-1)])
        r = 1/θ[2]
        p = 1 ./ (1  .+ θ[2] * μt)
        W[:,tt] = pdf.(NegativeBinomial.(r, p), Y.Ct[tt])
            
        # Resample
        inds = wsample(1:N, W[:,tt], N; replace=true)
        R[:, max(tt - L, 1):tt] = R[inds, max(tt - L, 1):tt]
        I[:, max(tt - L, 1):tt] = I[inds, max(tt - L, 1):tt]

        # Store expected cases if we are finding predictive posterior
        if opts["predictiveValues"]
            μ[:,tt] = μt
            μ[:,max(tt-L,1):tt] = μ[inds,max(tt-L,1):tt]
        end
                    
    end
    
    # Fetch predictive values
    if opts["predictiveValues"]
        for tt = 2:T
            r = 1/θ[2]
            p = 1 ./ (1 .+ θ[2] * μ[:,tt])
            C[:,tt] = rand.(NegativeBinomial.(r, p))
        end
    end
    
    # Run forecast
    if h > 0
        for tt = (T+1):(T+h)
            R[:,tt] = exp.(rand.(Normal.(log.(R[:,tt-1]), θ[1])))
            Λ = sum(I[:, (tt-1):-1:1] .* ω[1:(tt-1)]', dims=2) ./ sum(ω[1:(tt-1)])
            I[:,tt] = rand.(Poisson.(R[:,tt] .* Λ))
            μt = sum(I[:,(tt-1):-1:1] .* delayDist[1:(tt-1)]', dims=2) ./ sum(delayDist[1:(tt-1)])
            r = 1/θ[2]
            p = 1 ./ (1  .+ θ[2] * μt)
            C[:,tt] = rand.(NegativeBinomial.(r, p))
        end
    end
    
    # Store output as three-dimensional array
    X = zeros(N, T+h, 3)
    X[:,:,1] = R
    X[:,:,2] = I
    X[:,:,3] = C
    
    # Forecast
    return(X, W)
    
end


naiveModel (generic function with 1 method)

In [3]:
opts = Dict(

    # Bootstrap filter options
    "T" => size(Y, 1), # Number of time-steps
    "N" => 10000, # Number of particles
    "L" => 50, # Fixed-lag resampling length
    "ω" => pdf.(Gamma(2.36, 2.74), 1:128), # Serial interval
    "delayDist" => pdf.(Gamma(5.72, 0.96), 1:200), # Observation delay distribution
    "pR0" => Uniform(0, 10), # Prior on Rt at t = 0
    "pI0" => DiscreteUniform(200, 600), # Prior on I at t = 0
    "predictiveValues" => false, # Whether to calculate predictive cases
    "forecastingHorizon" => 0, # Number of days to forecast

    # PMMH options
    "nChains" => 3, # Number of chains
    "chunkSize" => 100, # Number of iterations
    "maxChunks" => 50, # Maximum number of chunks
    "maxRhat" => 1.05,  # Stopping criterion: maximum Rhat value
    "minESS" => 100, # Stopping criterion: minimum effective sample size
    "showChunkProgress" => true, # Whether to show progress of each chunk
    "propStdDevInit" => sqrt.([0.01, 0.001]), # Initial proposal standard deviation (this is adaptively fit)
    "paramPriors" => [Uniform(0, 1), Uniform(0, 0.1)],
    "initialParamSamplers" => [Uniform(0.05, 0.2), Uniform(0.01, 0.02)],
    "paramLimits" => [(0, 1), (0, 0.1)],
    "paramNames" => ["σ", "ϕ"],

    # Marginal posterior options
    "posteriorNumberOfParticles" => 10000,
    "posteriorParamSamples" => 100

);

In [5]:
# Run PMMH (algorithm 2)
(θ, diag) = PMMH(naiveModel, Y, opts; verbose=true)
chains = Chains(θ, opts["paramNames"])

Running chunk 1 100%|██████████████████████████████████████████████████| Time: 0:00:54


2×2 Matrix{Float64}:
  0.00187387  -8.39656e-5
 -8.39656e-5   0.000283946

Old frobenius: 0.010049875621120892, new frobenius: 0.0018989782600318994, change: 0.18895539921319948
2×7 DataFrame
 Row │ chunk  tuning  pAccept    Rhat     ESS      mean       std       
     │ Int64  Bool    Float64    Float64  Float64  Float64    Float64   
─────┼──────────────────────────────────────────────────────────────────
   1 │     1    true  0.0639731  1.04792  26.4624  0.082626   0.0415333
   2 │     1    true  0.0639731  1.22303  10.021   0.0687253  0.0151664


Running chunk 2 100%|██████████████████████████████████████████████████| Time: 0:01:12


2×2 Matrix{Float64}:
  0.00116843  -3.18631e-5
 -3.18631e-5   0.000205486

...

Frobenius of covariance matrix: 0.0007005981153009555
2×7 DataFrame
 Row │ chunk  tuning  pAccept   Rhat     ESS       mean       std        
     │ Int64  Bool    Float64   Float64  Float64   Float64    Float64    
─────┼───────────────────────────────────────────────────────────────────
   1 │     6   false  0.161616  1.04634  103.294   0.0638999  0.0182889
   2 │     6   false  0.161616  1.01293   76.5842  0.0726604  0.00944699


Running chunk 7 100%|██████████████████████████████████████████████████| Time: 0:01:20


2×2 Matrix{Float64}:
  0.000657541  -2.32663e-5
 -2.32663e-5    0.000130488

Frobenius of covariance matrix: 0.0006711707667446496
2×7 DataFrame
 Row │ chunk  tuning  pAccept   Rhat     ESS      mean       std       
     │ Int64  Bool    Float64   Float64  Float64  Float64    Float64   
─────┼─────────────────────────────────────────────────────────────────
   1 │     7   false  0.340067  1.01203  162.278  0.0648873  0.0191057
   2 │     7   false  0.340067  1.01543  110.041  0.0730798  0.009896


Chains MCMC chain (400×2×3 Array{Float64, 3}):

Iterations        = 1:1:400
Number of chains  = 3
Samples per chain = 400
parameters        = σ, ϕ

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat 
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64 

           σ    0.0649    0.0191     0.0006    0.0015   162.2780    1.0120
           ϕ    0.0731    0.0099     0.0003    0.0008   110.0407    1.0154

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

           σ    0.0385    0.0520    0.0623    0.0734    0.1159
           ϕ    0.0527    0.0670    0.0729    0.0815    0.0919


In [7]:
# Fetch marginal posterior
opts["predictiveValues"] = true
opts["forecastingHorizon"] = 28
X = marginalPosterior(naiveModel, θ, Y, opts);

In [8]:
# Save model results
using HDF5
if isfile("temp/naiveResults.h5")
    rm("temp/naiveResults.h5")
end
h5write("temp/naiveResults.h5", "X", X)
h5write("temp/naiveResults.h5", "theta", θ)